In [71]:
let square x = x * x;

val square : int -> int = <fun>


Ocaml中的函数也是值。

In [72]:
let sum_if_true test first second = (if test first then first else 0) + (if test second then second else 0)

val sum_if_true : (int -> bool) -> int -> int -> int = <fun>


In [73]:
let even x = x mod 2 = 0;;

val even : int -> bool = <fun>


In [74]:
sum_if_true even 3 4;;

- : int = 4


注意在`even`定义中，我们采用了两种不同的方式使用=: 一次是作为`let`绑定的一部分，将所定义的变量与它的定义分开；另一次是作为一个相等行测试，将`x mod 2`与`0`进行比较。尽管语法有共同之处，但他们是完全不同的操作。

Ocaml使用一种称为类型推断（type inference）的技术来确定一个表达式的类型。

为了便于更好的了解一个表达式的类型，可以增加显示的类型标注(`type annotations`)。这些标注不会影响Ocaml程序的行为，不过他们可以作为很有用的文档，还可以捕获不期望的类型改变。如果一段代码编译失败，要明确代码未成功编译的原因，类型标注也会很有帮助。

In [75]:
let first_if_true test x  y = if test x then x else y

val first_if_true : ('a -> bool) -> 'a -> 'a -> 'a = <fun>


Ocaml并没有查看顶层环境的类型，可以看到，Ocaml并没有选择某一个具体的类型，它引入了一个类型变量（type variable）。`'a`来表示这个类型是泛型。这种泛型特性被称为参数化多态。这与C#和Java中的泛型极其相似

元组是一个有序集合，其中的值是不同的类型。可以用逗号连接各个值来创建一个元组

In [76]:
let a_tuple = (3, "three")

val a_tuple : int * string = (3, "three")


可以使用Ocaml的模式匹配语言抽取元组中的分量，如下所示

In [77]:
let (x, y) = a_tuple

val x : int = 3
val y : string = "three"


In [78]:
List.length [1;2;3]

- : int = 3


In [79]:
let languages = ["Ocaml";"Perl";"C"]

val languages : string list = ["Ocaml"; "Perl"; "C"]


In [80]:

open Core;;

In [115]:
List.map languages ~f:String.length

- : int Core.List.t = [5; 4; 1]


需要指出，传入`List.map`的是通过一个标签函数(labeled argument)`~f`传递的。标签参数按名指定，而不是按位置指定，所以允许你改变函数参数的顺序，这不会改变它的行为。

除了使用中括号构造列表之外，还可以使用操作符`::`在列表前面添加元素

In [82]:
"French" :: "Spanish" :: languages

- : string list = ["French"; "Spanish"; "Ocaml"; "Perl"; "C"]


與其他語言不同，`Ocaml`使用分號而不是逗號來分割列表中的列表元素。逗號用來分割原組中的元素，如果你試圖在列表中使用逗號，景觀代碼會編譯，但是不會得到你預期的結果。

In [83]:
["Ocaml", "Perl", "C"]

- : (string * string * string) list = [("Ocaml", "Perl", "C")]


需要注意的是，與::不同，@並不是一個常數時間的操作。連接兩個列表的時間與第一個列表的長度成正比。

In [84]:
let my_favorite_language (my_favorite :: the_rest) = my_favorite

File "[84]", line 1, characters 25-64:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
[]


val my_favorite_language : 'a list -> 'a = <fun>


這是使用::的模式匹配，利用這個模式匹配，我們分離了列表的第一個元素和列表的其餘部分，並風別命名爲`my_favourite`和`the_rest`。

不過，可以看到，頂層環境並不喜歡這個定義，它會給出一個警告，更重要的是，這樣可以確保你的代碼能處理所有可能的情況。它會給出一個警告，指出模式不完善，沒有窮盡所有情況。這說明這個模式可能無法匹配某些類型的值，即`[]`（空表）。試着運行`my_favourite_language`時可以看到，對於非空列表它確實能正常工作，但對於空表則會失敗。

In [85]:
my_favorite_language ["English"; "Spanish"; "Franch"]
my_favorite_language []

error: compile_error

可以使用match來避免這些警告，更重要的是，這樣可以確保你的代碼能處理所有可能的情況

In [86]:
let my_favorite_language languages = 
  match languages with
  | first :: the_rest -> first
  | [] -> "Ocaml"
  

val my_favorite_language : string list -> string = <fun>


In [87]:
my_favorite_language ["English"; "Spanish"; "French"] 

- : string = "English"


In [88]:
my_favorite_language []

- : string = "Ocaml"


In [89]:
let rec sum l = 
  match l with
  | [] -> 0
  | hd :: tl -> hd + sum tl
  

val sum : int list -> int = <fun>


In [90]:
sum [1;2;3]

- : int = 6


In [91]:
let rec destutter list = 
  match list with
  | [] -> []
  | [hd] -> [hd]
  | hd1 :: hd2 :: t1-> 
     if hd1 = hd2 then destutter (hd2 :: t1)
     else hd1 :: destutter (hd2 :: t1)

val destutter : 'a list -> 'a list = <fun>


`in`標誌這個作用域的開始，新變量可以在這個作用域中使用

In [92]:
 let x = 7 in
  x + x

- : int = 14


let綁定的作用域用雙分號結束，所以`x`的值不再可用

In [93]:
x

- : int = 3


選項在Ocaml中很重要，因爲這是Ocaml中對可能不存在的值進行的標準方法。在`Ocaml`中沒有類似`NullPointerException`的概念，這一點與其大多數語言都有所不同，包括Java與C#，那些語言中，絕大多數數據類型都有可能爲null。在這些語言中，到處都潛伏着null。

不過，Ocaml中則完全不同，如果沒有值，就要明確的指出。類型爲`string  * string`的值總是包含兩個明確定義的String類型值。如果允許其中某個值爲空，例如，允許第一個值不存在，就要把類型改爲`string option * string`。在第七章我們可以看到，基於這種明確性，更有利於確保你正確的處理可能缺少數據的情況。

In [94]:
type point2d = {x: float; y: float}

type point2d = { x : float; y : float; }


point2d是一個記錄類型(record)，可以把它看作是一個元祖，只是其中各個自斷是命名字段，而不是按位置來定義。

In [95]:
let p = {x = 3.; y = -4.}

val p : point2d = {x = 3.; y = -4.}


In [96]:
let magnitude {x = x_pos; y = y_pos} = 
 sqrt (x_pos ** 2. +. y_pos ** 2.)

val magnitude : point2d -> float = <fun>


可以使用字段双关把它写得更简练。具体地，如果字段名和该字段要绑定的字段名一致，就不需要把它们写下来。还可以重写为以下形式

In [102]:
let magnitude { x;y } = sqrt (x ** 2. +. y ** 2.)

val magnitude : point2d -> float = <fun>


或者也可以使用点记法访问记录字段

In [104]:
let distance v1 v2 = magnitude {x = v1.x -. v2.x; y = v1.y -. v2.y}

val distance : point2d -> point2d -> float = <fun>


当然可以把新定义的类型作为分量包含在更大的类型中。例如，下面给出一些类型为不同的几何对象建模，其中要用到`point2d`类型的值

In [107]:
type circle_desc = {center: point2d; radius: float}
type rect_desc = {lower_left:point2d; width: float;height: float}
type segment_desc = {endpoint1: point2d; endpoint2: point2d}

type circle_desc = { center : point2d; radius : float; }


type rect_desc = { lower_left : point2d; width : float; height : float; }


type segment_desc = { endpoint1 : point2d; endpoint2 : point2d; }


In [113]:
type scene_element = 
| Circle of circle_desc
| Rect of rect_desc
| Segment of segment_desc

type scene_element =
    Circle of circle_desc
  | Rect of rect_desc
  | Segment of segment_desc


可以如下写一个函数，测试一个点是否落在`scene_element`列表中某个元素的内部

In [112]:
let is_inside_scene_element point scene_element =
match scene_element with
| Circle {center; radius} ->
 distance center point < radius
| Rect { lower_left; width; height} ->
point.x > lower_left.x && point.x < lower_left.x +. width
&& point.y > lower_left.y && point.y < lower_left.y +. height
| Segment {endpoint1; endpoint2} -> false
 

val is_inside_scene_element : point2d -> scene_element -> bool = <fun>


In [114]:
let is_inside_scene point scene = List.exists scene ~f:(fun el -> is_inside_scene_element point el)

val is_inside_scene : point2d -> scene_element Core.List.t -> bool = <fun>


In [121]:
is_inside_scene {x=3.;y=7.}
[ Circle {center = {x = 4.;y=4.}; radius=0.5}]

- : bool = false


In [123]:
is_inside_scene {x=3.;y=7.}
[ Circle {center = {x = 4.;y=4.}; radius=5.0}]

- : bool = true


你可能会注意到，这里使用的`match`与处理`option`和`list`时使用的`match`是很类似的。这不是偶然，`option`和`list`实际上就是变体类型的例子，只是由于它们及其重要，因此标准库中提供了专门的定义。

Ocaml中默认为函数式代码，变量绑定和大多数数据结构都是不可变的。

In [129]:
let numbers = [| 1;2;3;4 |]

val numbers : int array = [|1; 2; 3; 4|]


In [133]:
numbers.(2) <- 4

- : unit = ()


`unit`类型可能只有一个值，写为()。我们使用`unit`表示设置可变字段等操作的返回值，这些操作的意义在于其副作用而不是返回一个值。它还可以用作不需要输入值的函数参数。`unit`值就类似于C和Java中void的角色。


In [137]:
numbers

- : int array = [|1; 2; 4; 4|]


尽管记录(`record`)默认是不可变的，但是可以把一些字段显式声明为可变字段

In [139]:
type running_sum = 
{
mutable sum: float;
mutable sum_sq: float;
mutable samples: int;
}

type running_sum = {
  mutable sum : float;
  mutable sum_sq : float;
  mutable samples : int;
}


In [141]:
let mean rsum = rsum.sum /. float rsum.samples
let stdev rsum = sqrt (rsum.sum_sq /. float rsum.samples -. (rsum.sum /. float rsum.samples)** 2.);;

val mean : running_sum -> float = <fun>


val stdev : running_sum -> float = <fun>
